# Lab#3 From Training to Deployment (Student)


---



# Copy the dataset from Gdrive to Colab

In [ ]:
#!cp -rf drive/MyDrive/MediaTek_IEE5725_Machine_Learning_Lab3/ .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-gpu==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411.5 MB 7.3 kB/s 
     |████████████████████████████████| 503 kB 52.8 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5e690520730881d463661e54bc257a048daa5882fd7c75cb84ffc32160ccc037
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting u

# Unzip your dataset



In [ ]:
#!zip -s- /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset.zip -O /content/MediaTek_IEE5725_Machine_Learning_Lab3/COMBINED_FILE.zip
#!unzip /content/MediaTek_IEE5725_Machine_Learning_Lab3/COMBINED_FILE.zip -d /content/MediaTek_IEE5725_Machine_Learning_Lab3/
!unzip /content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset.zip -d /content/MediaTek_IEE5725_Machine_Learning_Lab3/

串流輸出內容已截斷至最後 5000 行。
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/64439bfa-b8306cad_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/6443abb9-cbae0436_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/64451ee1-863d234a_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/64451ee1-a59697e1_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/64451ee1-aace66a3_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/color_labels/64451ee1-eaa0fb8f_lane_line_color.png  
  inflating: /content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/col

In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import glob
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import tensorflow as tf

from pathlib import Path
from tensorflow.python.framework.graph_util import convert_variables_to_constants

In [ ]:
print("Tensorflow Version is %s" % tf.__version__)

Tensorflow Version is 1.15.0


# Data Process

In [ ]:
class DataLoaderSegmentation(data.Dataset):
    def __init__(self, input_path,label_path,label_name='',transform=None):
        super(DataLoaderSegmentation, self).__init__()
        self.img_files = glob.glob(os.path.join(input_path,'*.jpg'))
        self.mask_files = []
        self.transforms = transform
        for img_path in self.img_files:
             self.mask_files.append(os.path.join(label_path,os.path.basename(img_path).split('.')[0]+label_name+'.png'))

    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data = cv2.imread(img_path)
            label = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
            label = F.one_hot(torch.from_numpy(label).to(torch.int64),6)
            datalabel = np.concatenate((data,label),axis=2)
            datalabel = np.transpose(datalabel,[2,0,1])
            if self.transforms!=None:
              datalabel = self.transforms(torch.from_numpy(datalabel).float())
            datalabel = np.transpose(datalabel,[1,2,0])
            data = datalabel[:,:,0:3]
            label = datalabel[:,:,3:9]
            return data,label

    def __len__(self):
        return len(self.img_files)

# Prepare Training dataset

In [ ]:
input_path='/content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/images'#720/1280
label_path='/content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels/class_labels'
transform_list = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize(size=(720,1280))                                   
])
dataset = DataLoaderSegmentation(input_path,label_path,'_lane_line_label_id',transforms.Resize(size=(720,1280)))
#dataset = DataLoaderSegmentation(input_path,label_path,'_lane_line_label_id',transform_list)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=25, shuffle=True)

In [ ]:
input_path='/content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/images_real_world'#1080/1920
label_path='/content/MediaTek_IEE5725_Machine_Learning_Lab3/ICME2022_Training_Dataset/labels_real_world'
transform_list = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize(size=(1080,1920))                                   
])

dataset_real = DataLoaderSegmentation(input_path,label_path,'',transforms.Resize(size=(1080,1920)))
#dataset_real = DataLoaderSegmentation(input_path,label_path,'',transform_list)
dataloader_real = torch.utils.data.DataLoader(dataset_real, batch_size=25, shuffle=True)

# FIXME#1 Design the Network
*   input size = 256*256
*   Channel = 15 and Depth = 5
*   backbone: conv2d+BN+Relu
*   output size = 1080*1920





In [ ]:
inputs = tf.placeholder(tf.float32,shape=(None,None, None, 3))
y_ = tf.placeholder(tf.float32, [None,None, None,6]) # 6 class
x = tf.image.resize_images(inputs, (256, 256))
x = x/255.0
y = tf.image.resize_images(y_, (256, 256))
ch=13 #13
depth=7 #5
xn = []
b=tf.Variable(0.0)
x = tf.layers.conv2d(x,ch,3,1,'same')
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x)
for i in range(depth):
  xn.append(x)
  x = tf.layers.conv2d(x,ch*(2**(i+1)),3,1,'same')
  x = tf.layers.batch_normalization(x,center=False,scale=False)+b
  x = tf.nn.relu(x)
  x = tf.layers.conv2d(x,ch*(2**(i+1)),3,1,'same')
  x = tf.layers.batch_normalization(x,center=False,scale=False)+b
  x = tf.nn.relu(x)
  if i <depth-1:
    x = tf.nn.avg_pool(x,[1,2,2,1],[1,2,2,1],'SAME')
for i in range(depth):
  if i>0:
    x = tf.keras.layers.UpSampling2D((2,2))(x)
  x = tf.layers.conv2d(x,ch*(2**(depth-i-1)),3,1,'same')+xn[-i-1]
  x = tf.layers.batch_normalization(x,center=False,scale=False)+b
  x = tf.nn.relu(x)
out = tf.layers.conv2d(x,6,3,1,'same')
outputs = out
outputs = tf.image.resize_images(outputs, (1080, 1920))
outputs = tf.argmax(outputs,-1)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


# Set up the Hyper parameters

In [ ]:
loss=tf.nn.softmax_cross_entropy_with_logits_v2(logits=out,labels=y)
loss=tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate = 5e-4)
# optimizer = tf.train.MomentumOptimizer(learning_rate=0.0001, momentum=0.9, use_nesterov=True)
train = optimizer.minimize(loss+0.0005*b)
saver = tf.train.Saver()
init = tf.global_variables_initializer()

# Sesstion run and restore Checkpoint

In [ ]:
sess = tf.Session()
sess.run(init)
saver.restore(sess, '/content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/model/') 

INFO:tensorflow:Restoring parameters from /content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/model/


# Model Profiling to get Flops and #Parameters

In [ ]:
def stats_graph(graph):
  flops = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.float_operation())
  params = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.trainable_variables_parameter())
  print('FLOPs: {}; Trainable params:{}'.format(flops.total_float_ops, params.total_parameters))
stats_graph(tf.get_default_graph())

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


197 ops no flops stats due to incomplete shapes.


FLOPs: 132901067; Trainable params:66458802


197 ops no flops stats due to incomplete shapes.


# Set training Epochs, Print the training logs and Save your Checkpoint

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
  for i, data in enumerate(dataloader, 0):
    input = data[0].numpy()
    label = data[1].numpy()
    sess.run(train,feed_dict={inputs: input, y_: label})
    if i % 10 == 0:
      print("[%d/%d][%s/%d] loss: %.4f b: %.4f "\
            %(epoch+1, num_epochs, str(i).zfill(4), len(dataloader), sess.run(loss,feed_dict={inputs: input, y_: label}),sess.run(b)) )
    if i%300==0:
      print('checkpoint saved')
      for i, data in enumerate(dataloader_real, 0):
        input = data[0].numpy()
        label = data[1].numpy()
        sess.run(train,feed_dict={inputs: input, y_: label})
      saver.save(sess, '/content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/model/')
  saver.save(sess, '/content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/model/')
  print('checkpoint saved')

[1/5][0000/1420] loss: 0.0652 b: -0.0348 
checkpoint saved
[1/5][0010/1420] loss: 0.0593 b: -0.0345 
[1/5][0020/1420] loss: 0.0655 b: -0.0354 
[1/5][0030/1420] loss: 0.0542 b: -0.0348 
[1/5][0040/1420] loss: 0.0448 b: -0.0345 
[1/5][0050/1420] loss: 0.0537 b: -0.0342 
[1/5][0060/1420] loss: 0.0418 b: -0.0345 
[1/5][0070/1420] loss: 0.0586 b: -0.0349 
[1/5][0080/1420] loss: 0.0569 b: -0.0353 
[1/5][0090/1420] loss: 0.0567 b: -0.0357 
[1/5][0100/1420] loss: 0.0529 b: -0.0353 
[1/5][0110/1420] loss: 0.0506 b: -0.0349 
[1/5][0120/1420] loss: 0.0500 b: -0.0349 
[1/5][0130/1420] loss: 0.0469 b: -0.0345 
[1/5][0140/1420] loss: 0.0624 b: -0.0354 
[1/5][0150/1420] loss: 0.0594 b: -0.0361 
[1/5][0160/1420] loss: 0.0554 b: -0.0360 
[1/5][0170/1420] loss: 0.0551 b: -0.0363 
[1/5][0180/1420] loss: 0.0511 b: -0.0363 
[1/5][0190/1420] loss: 0.0468 b: -0.0359 
[1/5][0200/1420] loss: 0.0581 b: -0.0350 
[1/5][0210/1420] loss: 0.0542 b: -0.0349 
[1/5][0220/1420] loss: 0.0438 b: -0.0352 
[1/5][0230/1420] 

# Save your graph into .pb

In [ ]:
graph_def = convert_variables_to_constants(sess, sess.graph_def, output_node_names = ['ArgMax'])
tf.train.write_graph(graph_def, '/content/drive/Shareddrives/MTK_DATA/MediaTek_IEE5725_Machine_Learning_Lab3/', 'lab3_model.pb', as_text = False)

In [ ]:
for i, data in enumerate(dataloader_real, 0):
    if(i==1):
      break
      
show_id = 0
plt.imshow(data[0][show_id]/255.0)

In [ ]:
plt.imshow(np.argmax(data[1][show_id], 2))

# Show your results in different Epochs

In [ ]:
predict=sess.run(outputs,feed_dict={inputs: data[0]})
plt.imshow(predict[show_id])

In [ ]:
sess.close()

# Run your model on real device